In [80]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np
import json
from time import sleep, perf_counter as pf
import itertools as it

In [81]:
events = pd.read_csv('datasets/specialEvents.csv')
display(events)

,eventKey,name,isOffWork,type,outcome,startDate,endDate,startTime,endTime,city
0,0,Hockey,0,Sport,1.0,2014-10-08,2014-10-08,22:00:00,NaN,Vancouver
1,1,Hockey,1,Sport,1.0,2014-10-11,2014-10-11,22:00:00,NaN,Vancouver
2,2,Hockey,0,Sport,1.0,2014-10-17,2014-10-17,21:30:00,NaN,Vancouver
3,3,Hockey,1,Sport,2.0,2014-10-18,2014-10-18,22:00:00,NaN,Vancouver
4,4,Hockey,0,Sport,2.0,2014-10-21,2014-10-21,20:30:00,NaN,Vancouver
5,5,Hockey,0,Sport,1.0,2014-10-23,2014-10-23,20:00:00,NaN,Vancouver
6,6,Hockey,0,Sport,2.0,2014-10-24,2014-10-24,21:00:00,NaN,Vancouver
7,7,Hockey,1,Sport,1.0,2014-10-26,2014-10-26,21:30:00,NaN,Vancouver
8,8,Hockey,1,Sport,NaN,2014-10-26,2014-10-26,22:00:00,NaN,Vancouver
9,9,Hockey,0,Sport,1.0,2014-10-28,2014-10-28,22:00:00,NaN,Vancouver


In [82]:
display(events.loc[events['startDate'] != events['endDate']])

,eventKey,name,isOffWork,type,outcome,startDate,endDate,startTime,endTime,city
161,161,MeadowGrass Music Festival,0,Festival,NaN,2015-05-22,2015-05-24,NaN,NaN,Denver
177,177,Italian Heritage Month,0,Festival,NaN,2015-06-01,2015-06-30,NaN,NaN,Vancouver
178,178,City of Bhangra Festival 2015,0,Festival,NaN,2015-06-01,2015-06-06,NaN,NaN,Vancouver
184,184,Music Waste Festival,0,Festival,NaN,2015-06-04,2015-06-07,NaN,NaN,Vancouver
185,185,Bard on the Beach Shakespeare Festival,0,Festival,NaN,2015-06-04,2015-09-26,NaN,NaN,Vancouver
186,186,Pender Harbour Blues Festival,0,Festival,NaN,2015-06-05,2015-06-07,NaN,NaN,Vancouver
187,187,Surrey Greek Food Festival,0,Festival,NaN,2015-06-05,2015-06-14,NaN,NaN,Vancouver
194,194,Ladner Bandfest,1,Festival,NaN,2015-06-06,2015-06-07,NaN,NaN,Vancouver
195,195,Vancouver Mini Maker Faire,1,Festival,NaN,2015-06-06,2015-06-07,NaN,NaN,Vancouver
204,204,BC Renaissance Festival,0,Festival,NaN,2015-06-11,2015-06-14,NaN,NaN,Vancouver


In [83]:
output = []
for row in events.itertuples():
    for date in pd.date_range(row.startDate, row.endDate):
        output.append({
            'eventKey': row.eventKey,
            'name': row.name,
            'isOffWork': row.isOffWork,
            'type': row.type,
            'outcome': row.outcome,
            'startDate': row.startDate,
            'endDate': row.endDate,
            'startTime': row.startTime,
            'endTime': row.endTime,
            'city': row.city,
            'date': date
        })
df = pd.DataFrame(output)
df = df[['eventKey', 'name', 'isOffWork', 'type', 'outcome', 'startDate', 'endDate', 'startTime', 'endTime', 'date', 'city']]
display(df)

,eventKey,name,isOffWork,type,outcome,startDate,endDate,startTime,endTime,date,city
0,0,Hockey,0,Sport,1.0,2014-10-08,2014-10-08,22:00:00,NaN,2014-10-08,Vancouver
1,1,Hockey,1,Sport,1.0,2014-10-11,2014-10-11,22:00:00,NaN,2014-10-11,Vancouver
2,2,Hockey,0,Sport,1.0,2014-10-17,2014-10-17,21:30:00,NaN,2014-10-17,Vancouver
3,3,Hockey,1,Sport,2.0,2014-10-18,2014-10-18,22:00:00,NaN,2014-10-18,Vancouver
4,4,Hockey,0,Sport,2.0,2014-10-21,2014-10-21,20:30:00,NaN,2014-10-21,Vancouver
5,5,Hockey,0,Sport,1.0,2014-10-23,2014-10-23,20:00:00,NaN,2014-10-23,Vancouver
6,6,Hockey,0,Sport,2.0,2014-10-24,2014-10-24,21:00:00,NaN,2014-10-24,Vancouver
7,7,Hockey,1,Sport,1.0,2014-10-26,2014-10-26,21:30:00,NaN,2014-10-26,Vancouver
8,8,Hockey,1,Sport,NaN,2014-10-26,2014-10-26,22:00:00,NaN,2014-10-26,Vancouver
9,9,Hockey,0,Sport,1.0,2014-10-28,2014-10-28,22:00:00,NaN,2014-10-28,Vancouver


In [84]:
display(df.loc[df['startDate'] != df['endDate']])

,eventKey,name,isOffWork,type,outcome,startDate,endDate,startTime,endTime,date,city
161,161,MeadowGrass Music Festival,0,Festival,NaN,2015-05-22,2015-05-24,NaN,NaN,2015-05-22,Denver
162,161,MeadowGrass Music Festival,0,Festival,NaN,2015-05-22,2015-05-24,NaN,NaN,2015-05-23,Denver
163,161,MeadowGrass Music Festival,0,Festival,NaN,2015-05-22,2015-05-24,NaN,NaN,2015-05-24,Denver
179,177,Italian Heritage Month,0,Festival,NaN,2015-06-01,2015-06-30,NaN,NaN,2015-06-01,Vancouver
180,177,Italian Heritage Month,0,Festival,NaN,2015-06-01,2015-06-30,NaN,NaN,2015-06-02,Vancouver
181,177,Italian Heritage Month,0,Festival,NaN,2015-06-01,2015-06-30,NaN,NaN,2015-06-03,Vancouver
182,177,Italian Heritage Month,0,Festival,NaN,2015-06-01,2015-06-30,NaN,NaN,2015-06-04,Vancouver
183,177,Italian Heritage Month,0,Festival,NaN,2015-06-01,2015-06-30,NaN,NaN,2015-06-05,Vancouver
184,177,Italian Heritage Month,0,Festival,NaN,2015-06-01,2015-06-30,NaN,NaN,2015-06-06,Vancouver
185,177,Italian Heritage Month,0,Festival,NaN,2015-06-01,2015-06-30,NaN,NaN,2015-06-07,Vancouver


In [85]:
df.to_csv('datasets/events_mapping.csv', index=False)

In [86]:
df.drop(['city', 'date'], axis=1, inplace=True)
df.drop_duplicates(inplace=True)
display(df)
df.to_csv('datasets/final/events.csv', index=False)

,eventKey,name,isOffWork,type,outcome,startDate,endDate,startTime,endTime
0,0,Hockey,0,Sport,1.0,2014-10-08,2014-10-08,22:00:00,NaN
1,1,Hockey,1,Sport,1.0,2014-10-11,2014-10-11,22:00:00,NaN
2,2,Hockey,0,Sport,1.0,2014-10-17,2014-10-17,21:30:00,NaN
3,3,Hockey,1,Sport,2.0,2014-10-18,2014-10-18,22:00:00,NaN
4,4,Hockey,0,Sport,2.0,2014-10-21,2014-10-21,20:30:00,NaN
5,5,Hockey,0,Sport,1.0,2014-10-23,2014-10-23,20:00:00,NaN
6,6,Hockey,0,Sport,2.0,2014-10-24,2014-10-24,21:00:00,NaN
7,7,Hockey,1,Sport,1.0,2014-10-26,2014-10-26,21:30:00,NaN
8,8,Hockey,1,Sport,NaN,2014-10-26,2014-10-26,22:00:00,NaN
9,9,Hockey,0,Sport,1.0,2014-10-28,2014-10-28,22:00:00,NaN


In [87]:
df.loc[df['eventKey'] == 161]

,eventKey,name,isOffWork,type,outcome,startDate,endDate,startTime,endTime
161,161,MeadowGrass Music Festival,0,Festival,NaN,2015-05-22,2015-05-24,NaN,NaN
